In [1]:
from ipynb.fs.full.sim import Simulator
from ipynb.fs.full.traffic import Frame
from ipynb.fs.full.logger import Logger
from ipynb.fs.full.topology import SinkSwitch, BasicSwitch, Link, TrafficSource
from ipynb.fs.full.frer import FRERSwitch, FRERMerge, FRERSplit

In [2]:
sim = Simulator()
logger = Logger()

# Switches
sw1 = BasicSwitch("SW1", sim, logger, routing_table={"S1": "out1", "S2": "out2"})
sw1.add_input_port("in1")
sw1.add_input_port("in2")
sw1.add_output_port("out1", None)
sw1.add_output_port("out2", None)

sw2 = BasicSwitch("SW2", sim, logger, routing_table={"S1": "out1"})
sw2.add_input_port("in1")
sw2.add_input_port("in2")
sw2.add_output_port("out1", None)
sw2.add_output_port("out2", None)

    # Sinks
sink1 = SinkSwitch("SINK1", sim, logger)
sink1.add_input_port("in")

sink2 = SinkSwitch("SINK2", sim, logger)
sink2.add_input_port("in")


# Links
link_1_2 = Link(sim, delay=1.0, target_switch=sw2, target_port="in1")
sw1.output_ports["out1"] = link_1_2

link_1_sink1 = Link(sim, delay=1.0, target_switch=sink1, target_port="in")
sw1.output_ports["out2"] = link_1_sink1

link_2_sink2 = Link(sim, delay=1.0, target_switch=sink2, target_port="in")
sw2.output_ports["out1"] = link_2_sink2

# Traffic
source1 = TrafficSource(sim)
source1.start_stream(
    stream_id="S1",
    out_link=Link(sim, 0.0, sw1, "in1"),
    interval=1.0,
    size=100,
    count=5
)

source2 = TrafficSource(sim)
source2.start_stream(
    stream_id="S2",
    out_link=Link(sim, 0.0, sw1, "in2"),
    interval=2.0,
    size=100,
    count=3
)

sim.run(until=20)
logger.to_dataframe()

,time,node,event,frame_id,stream_id,port
0,0.0,SW1,RECEIVE,1,S1,in1
1,0.0,SW1,FORWARD,1,S1,out1
2,0.0,SW1,RECEIVE,1,S2,in2
3,0.0,SW1,FORWARD,1,S2,out2
4,1.0,SW2,RECEIVE,1,S1,in1
5,1.0,SW2,FORWARD,1,S1,out1
6,1.0,SINK1,RECEIVE,1,S2,in
7,1.0,SINK1,SINK,1,S2,in
8,1.0,SW1,RECEIVE,2,S1,in1
9,1.0,SW1,FORWARD,2,S1,out1


In [4]:
sim = Simulator()
logger = Logger()

# Switches
sw1 = BasicSwitch("SW1", sim, logger, routing_table={"S1": "out1", "S2": "out2"})
sw1.add_input_port("in1")
sw1.add_input_port("in2")
sw1.add_output_port("out1", None)
sw1.add_output_port("out2", None)

sw2 = BasicSwitch("SW2", sim, logger, routing_table={"S1": "out1", "S2": "out2"})
sw2.add_input_port("in1")
sw2.add_input_port("in2")
sw2.add_output_port("out1", None)
sw2.add_output_port("out2", None)

    # Sinks
sink1 = SinkSwitch("SINK1", sim, logger)
sink1.add_input_port("in")

sink2 = SinkSwitch("SINK2", sim, logger)
sink2.add_input_port("in")

    # FRER
split = FRERSplit({"S1": ["out1", "out2"]})
sw_split = FRERSwitch("SW_SPLIT", sim, logger, routing_table={}, frer_split=split)
sw_split.add_input_port("in1")
sw_split.add_input_port("in2")
sw_split.add_output_port("out1", None)
sw_split.add_output_port("out2", None)

merge = FRERMerge(expire_time=100.0)
sw_merge = FRERSwitch("SW_MERGE", sim, logger, routing_table={"S1": "out1"}, frer_merge=merge)
sw_merge.add_input_port("in1")
sw_merge.add_input_port("in2")
sw_merge.add_output_port("out1", None)
sw_merge.add_output_port("out2", None)

# Links
link_split_1 = Link(sim, delay=1.0, target_switch=sw1, target_port="in1")
sw_split.output_ports["out1"] = link_split_1

link_split_2 = Link(sim, delay=1.0, target_switch=sw2, target_port="in1")
sw_split.output_ports["out2"] = link_split_2

link_1_merge = Link(sim, delay=1.0, target_switch=sw_merge, target_port="in1")
sw1.output_ports["out1"] = link_1_merge

link_2_merge = Link(sim, delay=1.0, target_switch=sw_merge, target_port="in2")
sw2.output_ports["out1"] = link_2_merge

link_merge_sink = Link(sim, delay=1.0, target_switch=sink1, target_port="in")
sw_merge.output_ports["out1"] = link_merge_sink

# Traffic
source1 = TrafficSource(sim)
source1.start_stream(
    stream_id="S1",
    out_link=Link(sim, 0.0, sw_split, "in1"),
    interval=1.0,
    size=100,
    count=2
)

sim.run(until=20)
logger.to_dataframe()

,time,node,event,frame_id,stream_id,port
0,0.0,SW_SPLIT,RECEIVE,1,S1,in1
1,0.0,SW_SPLIT,FORWARD,1,S1,out1
2,0.0,SW_SPLIT,FORWARD,1,S1,out2
3,1.0,SW1,RECEIVE,1,S1,in1
4,1.0,SW1,FORWARD,1,S1,out1
5,1.0,SW2,RECEIVE,1,S1,in1
6,1.0,SW2,FORWARD,1,S1,out1
7,1.0,SW_SPLIT,RECEIVE,2,S1,in1
8,1.0,SW_SPLIT,FORWARD,2,S1,out1
9,1.0,SW_SPLIT,FORWARD,2,S1,out2
